<a href="https://colab.research.google.com/github/sanvidhaharibhakta/Practicals/blob/main/LargeVectorAddition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-n8um7842
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-n8um7842
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=75e3c220e1b516c0af3e4eb5b3aa4ebb789de87cbc1b9842b976f4f91da9cb9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-b5c6y402/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [1]:
%%writefile add.cu
#include <iostream>
#include <cstdlib> // Include <cstdlib> for rand()
using namespace std;
__global__
void add(int* A, int* B, int* C, int size) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < size) {
C[tid] = A[tid] + B[tid];
}
}
void print(int* vector, int size) {
for (int i = 0; i < size; i++) {
cout << vector[i] << " ";
}
cout << endl;
}
int main() {
int N;
cout << "Enter the size of the vectors: ";
cin >> N;
int* A, * B, * C;
int vectorSize = N;
size_t vectorBytes = vectorSize * sizeof(int);
// Allocate host memory
A = new int[vectorSize];
B = new int[vectorSize];
C = new int[vectorSize];
// Initialize host arrays
cout << "Enter elements of vector A:" << endl;
for (int i = 0; i < N; i++) {
cin >> A[i];
}
cout << "Enter elements of vector B:" << endl;
for (int i = 0; i < N; i++) {
cin >> B[i];
}
cout << "Vector A: ";
print(A, N);
cout << "Vector B: ";
print(B, N);
int* X, * Y, * Z;
// Allocate device memory
cudaMalloc(&X, vectorBytes);
cudaMalloc(&Y, vectorBytes);
cudaMalloc(&Z, vectorBytes);
// Check for CUDA memory allocation errors
if (X == nullptr || Y == nullptr || Z == nullptr) {
cerr << "CUDA memory allocation failed" << endl;
return 1;
}
// Copy data from host to device
cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);
int threadsPerBlock = 256;
int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
// Launch kernel
add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);
// Check for kernel launch errors
cudaError_t kernelLaunchError = cudaGetLastError();
if (kernelLaunchError != cudaSuccess) {
cerr << "CUDA kernel launch failed: " <<
cudaGetErrorString(kernelLaunchError) << endl;
return 1;
}
// Copy result from device to host
cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);
// Check for CUDA memcpy errors
cudaError_t memcpyError = cudaGetLastError();
if (memcpyError != cudaSuccess) {
cerr << "CUDA memcpy failed: " << cudaGetErrorString(memcpyError) <<
endl;
return 1;
}
cout << "Addition: ";
print(C, N);
// Free device memory
cudaFree(X);
cudaFree(Y);
cudaFree(Z);
// Free host memory
delete[] A;
delete[] B;
delete[] C;
return 0;
}

Writing add.cu


In [2]:
!nvcc add.cu -o add

In [3]:
!./add

Enter the size of the vectors: 7
Enter elements of vector A:
1
2
3
4
5
6
7
Enter elements of vector B:
0
9
8
7
6
5
4
Vector A: 1 2 3 4 5 6 7 
Vector B: 0 9 8 7 6 5 4 
Addition: 1 11 11 11 11 11 11 
